In [1]:
import os,sys
sys.path.append("%s/analysis-CommonTools/"%os.getenv('HOME'))
from common_imports import *
from matplotlib_tools import *

In [2]:
import ROOT

Welcome to JupyROOT 6.26/00


In [3]:
%jsroot on

In [29]:
def prepare_df(df,selection):
    
    df_sel = df.Filter(selection)
    
    df_sel = df_sel.Define("angle_azimuth","atan(dir.x/dir.z)")
    df_sel = df_sel.Define("angle_zenith","atan(sqrt(dir.x*dir.x+dir.y*dir.y)/dir.z)")

    df_planes = {}
    
    for p in range(3):
        #hitstr="hits%d"%plane
        df_sel = df_sel.Define(f"hits{p}_n",
                               f"hits{p}.dir.x.size()")
        df_planes[p] = df_sel.Filter(f"hits{p}_n > 0")
        df_planes[p] = df_planes[p].Define(f"hits{p}_angle_azimuth",
                                           f"atan(hits{p}.dir.x.back()/hits{p}.dir.z.back())")
        df_planes[p] = df_planes[p].Define(f"hits{p}_angle_zenith",
                                           f"atan(sqrt(hits{p}.dir.x.back()*hits{p}.dir.x.back()+hits{p}.dir.y.back()*hits{p}.dir.y.back())/hits{p}.dir.z.back())")
        for var in ["dqdx"]:
            df_planes[p] = df_planes[p].Define(f"hits{p}_{var}_mean",
                                               f"ROOT::VecOps::Mean(hits{p}.{var})")
        for hvar in ["integral","width"]:
            df_planes[p] = df_planes[p].Define(f"hits{p}_{hvar}_mean",
                                               f"ROOT::VecOps::Mean(hits{p}.h.{hvar})")
    
    return df_sel, df_planes

In [30]:
def make_histos_dict(df,selection):
    
    df_sel, df_sel_planes = prepare_df(df,selection)
    
    hists = {}
    
    for c in ["x","y","z"]:
        hists[f"start_{c}"] = df_sel.Histo1D(f"start.{c}")
        hists[f"end_{c}"] = df_sel.Histo1D(f"end.{c}")
        hists[f"dir_{c}"] = df_sel.Histo1D(f"dir.{c}")
    
    hists["length"] = df_sel.Histo1D("length")
    hists["t0"] = df_sel.Histo1D("t0")
    hists["angle_azimuth"] = df_sel.Histo1D("angle_azimuth")
    hists["angle_zenith"] = df_sel.Histo1D("angle_zenith")
    
    for p in range(3):
        for v in ["dqdx_mean","integral_mean","width_mean","angle_azimuth","angle_zenith","n"]:
            hists[f"hits{p}_{v}"] = df_sel_planes[p].Histo1D(f"hits{p}_{v}")
    
    return hists            

In [31]:
bnb_sim_files_dir = "/Users/wketchum/Data/ICARUS/CalibrationNtuples_10Mar2022/ICARUS_BNB_Nu_Cosmics"
bnb_data_files_dir = "/Users/wketchum/Data/ICARUS/CalibrationNtuples_10Mar2022/ICARUS_BNB_Data"

In [32]:
c = ROOT.TCanvas()

In [33]:
selection_stopping = "selected==0"

In [34]:
files_bnb_sim = ROOT.TFileCollection()
files_bnb_sim.Add(f"{bnb_sim_files_dir}/hist_prodcorsika_bnb*.root")
print("Found %d BNB sim files in our list."%files_bnb_sim.GetNFiles())

Found 11 BNB sim files in our list.


In [35]:
files_bnb_data = ROOT.TFileCollection()
files_bnb_data.Add(f"{bnb_data_files_dir}/hist_data_dl4_fstrmBNB*.root")
print("Found %d BNB data files in our list."%files_bnb_data.GetNFiles())

Found 27 BNB data files in our list.


In [36]:
chain_bnb_sim = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_sim.AddFileInfoList(files_bnb_sim.GetList())

1

In [37]:
chain_bnb_data = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_data.AddFileInfoList(files_bnb_data.GetList())

1

In [38]:
#df_bnb_sim_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_sim),selection_stopping)
hists_bnb_sim_stopping = make_histos_dict(ROOT.RDataFrame(chain_bnb_sim),selection_stopping)

In [40]:
#df_bnb_data_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_data),selection_stopping)
hists_bnb_data_stopping = make_histos_dict(ROOT.RDataFrame(chain_bnb_data),selection_stopping)

In [15]:
#ROOT.RDF.SaveGraph(hists_bnb_sim_stopping["hits2_dqdx_mean"])

In [39]:
hists_bnb_sim_stopping["hits2_dqdx_mean"].Draw()
c.Draw()

In [41]:
hists_bnb_data_stopping["hits2_dqdx_mean"].Draw()
c.Draw()

In [42]:
hists_bnb_data_stopping["hits1_dqdx_mean"].Draw()
c.Draw()